In [10]:
import torch
import matplotlib.pyplot as plt
import torch_dct as dct
from lib.tools import *
from lib.decomposition import *
from lib.jpeg import *
from lib.sketchml import *
from lib.regression import *
import os
import pandas as pd

In [2]:
# tensor_dir = "../experiments_football/tensors/"
tensor_dir = "./test_tensors/"
output_dir = "./measurements/"
layer = 2

test_tensors = os.listdir(tensor_dir)

In [3]:
## Measurements
split_layer = []

tensor_rank=[]
tensor_sparsity=[]
tensor_regularity=[]
tensor_pictoriality=[]

jpeg_ratio=[]
jpeg_snr=[]
jpeg_mse=[]
decomposition_ratio=[]
decomposition_snr=[]
decomposition_mse=[]
regression_ratio=[]
regression_snr=[]
regression_mse=[]
sketchml_ratio=[]
sketchml_snr=[]
sketchml_mse=[]

In [ ]:
for tensor_name in test_tensors:
    print("************* Testing tensor: "+tensor_name+"***************")
    split_layer.append(layer)
    tensor = torch.load(tensor_dir+tensor_name)
    tensor = tensor[0]

    tensor_rank.append(calculate_cp_rank(tensor.numpy(),max_rank=10))
    tensor_sparsity.append(calculate_sparsity(tensor.numpy()))
    tensor_pictoriality.append(get_tensor_pictoriality(tensor))
    tensor_regularity.append(get_tensor_regularity(tensor))
    print("Rank: %f, Sparsity: %f, Picoriality: %f, Regularity: %f" % (tensor_rank[-1],tensor_sparsity[-1],tensor_pictoriality[-1],tensor_regularity[-1]))

    ## Jpeg compression
    print("Doing JPEG compression")
    normalize_base, scale,zero_point, encoded_data, compressed_size_jpeg = compressor_jpeg_gray(tensor, 100)
    reconstructed_tensor_jpeg = decompressor_jpeg_gray(tensor.shape,normalize_base, scale,zero_point, encoded_data )
    ratio_jpeg = tensor.numel()*4/compressed_size_jpeg
    snr_jpeg = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_jpeg.numpy())
    mse_jpeg = calculate_mse(tensor.numpy(), reconstructed_tensor_jpeg.numpy())
    jpeg_ratio.append(ratio_jpeg)
    jpeg_snr.append(snr_jpeg)
    jpeg_mse.append(mse_jpeg)
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_jpeg,snr_jpeg,mse_jpeg))

    print("Doing Decomposition compression")
    factors, compressed_size_decomposition = compressor_decomposition(tensor, tensor_rank[-1])
    reconstructed_tensor_decomposition = decompressor_decomposition(factors)
    snr_decomposition = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_decomposition)
    mse_decomposition = calculate_mse(tensor.numpy(), reconstructed_tensor_decomposition)
    ratio_decomposition= tensor.numel()*4/compressed_size_decomposition
    decomposition_ratio.append(ratio_decomposition)
    decomposition_snr.append(snr_decomposition)
    decomposition_mse.append(mse_decomposition)
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_decomposition,snr_decomposition,mse_decomposition))

    print("Doing Regression compression")
    factors, x_pos, x_neg, compressed_size_regression = compressor_regression(tensor, 10)
    reconstructed_tensor_regression = decompressor_regression(tensor.shape, factors, x_pos,x_neg )
    snr_regression = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_regression.numpy())
    mse_regression = calculate_mse(tensor.numpy(), reconstructed_tensor_regression.numpy())
    ratio_regression = tensor.numel()*4 / compressed_size_regression
    regression_ratio.append(ratio_regression)
    regression_snr.append(snr_regression)
    regression_mse.append(mse_regression)
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_regression,snr_regression,mse_regression))

    print("Doing SketchML compression")
    minmaxsketch,bucket_means, keys, compressed_size_sketchml = compressor_sketchml(tensor,128,0.3,3)
    reconstructed_tensor_sketchml = decompressor_sketchml(tensor.shape, minmaxsketch,bucket_means, keys)
    snr_sketchml = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_sketchml.numpy())
    mse_sketchml = calculate_mse(tensor.numpy(), reconstructed_tensor_sketchml.numpy())
    ratio_sketchml = tensor.numel() *4 / compressed_size_sketchml
    sketchml_ratio.append(ratio_sketchml)
    sketchml_snr.append(snr_sketchml)
    sketchml_mse.append(mse_sketchml)
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_sketchml,snr_sketchml,mse_sketchml))


In [11]:
dict = {'split_layer':split_layer,
        'rank': tensor_rank, 
        'sparsity': tensor_sparsity, 
        'regularity': tensor_regularity,
        'pictoriality': tensor_pictoriality,
        'jpeg_ratio':jpeg_ratio,
        'jpeg_snr':jpeg_snr,
        'jpeg_mse':jpeg_mse,
        'decomposition_ratio':decomposition_ratio,
        'decomposition_snr':decomposition_snr,
        'decomposition_mse':decomposition_mse,
        'regression_ratio':regression_ratio,
        'regression_snr':regression_snr,
        'regression_mse':regression_mse,
        'sketchml_ratio':sketchml_ratio,
        'sketchml_snr':sketchml_snr,
        'sketchml_mse':sketchml_mse
        } 
   
df = pd.DataFrame(dict)

In [13]:
df.to_csv(output_dir+"layer2.csv",index=False, header=True)